<a href="https://colab.research.google.com/github/wanlichen2024/612testing1/blob/main/data_storage_SQLite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# create database at SQLite

In [1]:
import sqlite3

# connect to sqlite
# create a database if database does not exists
conn = sqlite3.connect('environmental_data.db')

# create a cursor
cursor = conn.cursor()

# create species table
create_species_table = '''
CREATE TABLE IF NOT EXISTS Species (
    Species_ID INTEGER PRIMARY KEY,
    Species_Name TEXT NOT NULL
)
'''

# create source table
create_source_table = '''
CREATE TABLE IF NOT EXISTS Source (
    Source_ID INTEGER PRIMARY KEY,
    Source_Name TEXT NOT NULL
)
'''

# create sample table
create_samples_table = '''
CREATE TABLE IF NOT EXISTS Samples (
    Sample_ID INTEGER PRIMARY KEY,
    Date TEXT NOT NULL,
    Time TEXT NOT NULL,
    Location TEXT NOT NULL
)
'''

# create measurement table
create_measurement_table = '''
CREATE TABLE IF NOT EXISTS Measurement (
    Measurement_ID INTEGER PRIMARY KEY,
    Species_ID INTEGER,
    Source_ID INTEGER,
    Sample_ID INTEGER,
    Concentration REAL,
    Average REAL,
    Error REAL,
    Dispersion REAL,
    Envelope REAL,
    Exceedance REAL,
    FOREIGN KEY (Species_ID) REFERENCES Species (Species_ID),
    FOREIGN KEY (Source_ID) REFERENCES Source (Source_ID),
    FOREIGN KEY (Sample_ID) REFERENCES Samples (Sample_ID)
)
'''

# exucution of creating table
cursor.execute(create_species_table)
cursor.execute(create_source_table)
cursor.execute(create_samples_table)
cursor.execute(create_measurement_table)

# commitment
conn.commit()

# close
cursor.close()
conn.close()

# Insert data into database

In [3]:
import sqlite3
import csv

# 连接到SQLite数据库
conn = sqlite3.connect('environmental_data.db')
cursor = conn.cursor()

# 读取CSV文件并插入数据
def insert_data(cursor, table, columns, data):
    sql = f'INSERT INTO {table} ({", ".join(columns)}) VALUES ({", ".join(["?" for _ in columns])})'
    cursor.executemany(sql, data)

# 读取CSV文件
def read_csv(filename):
    with open(filename, newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        data = [row for row in reader]
    return data

# 准备插入数据的列名和数据
species_columns = ['Species_ID', 'Species_Name']
source_columns = ['Source_ID', 'Source_Name']
samples_columns = ['Sample_ID', 'Date', 'Time', 'Location']
measurement_columns = ['Measurement_ID', 'Species_ID', 'Source_ID', 'Sample_ID', 'Concentration', 'Average', 'Error', 'Dispersion', 'Envelope', 'Exceedance']

# 假设CSV文件中的数据已经是按照Species分组的
species_data = read_csv('/content/drive/MyDrive/713 Capstone Project (DA)/PMF_Raw Date_QueenSt_PM10_1.csv')

# 插入Species数据
# 注意：这里需要根据实际CSV文件的结构来调整代码
for species in species_data:
    cursor.execute(f'INSERT INTO Species (Species_Name) VALUES (?)', (species['Species'],))
    species_id = cursor.lastrowid
    # 假设每个Species下的数据是连续的
    for row in species_data[species_data.index(species):]:
        if row['Species'] == species['Species']:
            insert_data(cursor, 'Source', source_columns, [(row['Source_Name'],)])
            source_id = cursor.lastrowid
            # 插入Measurement数据
            insert_data(cursor, 'Measurement', measurement_columns, [(row['Concentration'], row['Average'], row['Error'], row['Dispersion'], row['Envelope'], row['Exceedance'], species_id, source_id, 1)])  # 假设Sample_ID为1

# 提交事务
conn.commit()

# 关闭游标和连接
cursor.close()
conn.close()

KeyError: 'Source_Name'